⚠️ This project is mandatory for certification bloc #1.

![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# Plan your trip with Kayak 

## Company's description 📇

<a href="https://www.kayak.com" target="_blank">Kayak</a> is a travel search engine that helps user plan their next trip at the best price.

The company was founded in 2004 by Steve Hafner & Paul M. English. After a few rounds of fundraising, Kayak was acquired by <a href="https://www.bookingholdings.com/" target="_blank">Booking Holdings</a> which now holds: 

* <a href="https://booking.com/" target="_blank">Booking.com</a>
* <a href="https://kayak.com/" target="_blank">Kayak</a>
* <a href="https://www.priceline.com/" target="_blank">Priceline</a>
* <a href="https://www.agoda.com/" target="_blank">Agoda</a>
* <a href="https://Rentalcars.com/" target="_blank">RentalCars</a>
* <a href="https://www.opentable.com/" target="_blank">OpenTable</a>

With over \$300 million revenue a year, Kayak operates in almost all countries and all languages to help their users book travels accros the globe. 

## Project 🚧

The marketing team needs help on a new project. After doing some user research, the team discovered that **70% of their users who are planning a trip would like to have more information about the destination they are going to**. 

In addition, user research shows that **people tend to be defiant about the information they are reading if they don't know the brand** which produced the content. 

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

* Weather 
* Hotels in the area 

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time. 

## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

## Scope of this project 🖼️

Marketing team wants to focus first on the best cities to travel to in France. According <a href="https://one-week-in.com/35-cities-to-visit-in-france/" target="_blank">One Week In.com</a> here are the top-35 cities to visit in France: 

```python 
["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]
```

Your team should focus **only on the above cities for your project**. 


## Helpers 🦮

To help you achieve this project, here are a few tips that should help you

### Get weather data with an API 

*   Use https://nominatim.org/ to get the gps coordinates of all the cities (no subscription required) Documentation : https://nominatim.org/release-docs/develop/api/Search/

*   Use https://openweathermap.org/appid (you have to subscribe to get a free apikey) and https://openweathermap.org/api/one-call-api to get some information about the weather for the 35 cities and put it in a DataFrame

*   Determine the list of cities where the weather will be the nicest within the next 7 days For example, you can use the values of daily.pop and daily.rain to compute the expected volume of rain within the next 7 days... But it's only an example, actually you can have different opinions on a what a nice weather would be like 😎 Maybe the most important criterion for you is the temperature or humidity, so feel free to change the rules !

*   Save all the results in a `.csv` file, you will use it later 😉 You can save all the informations that seem important to you ! Don't forget to save the name of the cities, and also to create a column containing a unique identifier (id) of each city (this is important for what's next in the project)

*   Use plotly to display the best destinations on a map

### Scrape Booking.com 

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com 

You can scrap as many information asyou want, but we suggest that you get at least:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel


### Create your data lake using S3 

Once you managed to build your dataset, you should store into S3 as a csv file. 

### ETL 

Once you uploaded your data onto S3, it will be better for the next data analysis team to extract clean data directly from a Data Warehouse. Therefore, create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB. 

## Deliverable 📬

To complete this project, your team should deliver:

* A `.csv` file in an S3 bucket containing enriched information about weather and hotels for each french city

* A SQL Database where we should be able to get the same cleaned data from S3 

* Two maps where you should have a Top-5 destinations and a Top-20 hotels in the area. You can use plotly or any other library to do so. It should look something like this: 

![Map](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/Kayak_best_destination_project.png)

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
from tqdm import tqdm
import datetime as dt
api_key='25143b0395a1953fbdcfd568cbd5c928'

In [2]:
cities =["Mont-Saint-Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

In [3]:
#on crée une lsite vide ou l'on stock les requests en bouclant sur notre liste de villes, on sort au format json
liste=[]
for i in tqdm(cities):
    r=requests.get('https://nominatim.openstreetmap.org/?addressdetails=1&q=' +str(i) +'&countrycodes=france&format=json&limit=1')
    liste.append(r.json())

100%|██████████| 35/35 [00:17<00:00,  2.04it/s]


In [4]:
#on obtient une liste de dictionnaire mais contenant un dictionnaire donc on flatten chaque dict
for i in range(0,len(liste)):
    liste[i][0].update(liste[i][0].pop('address'))

In [5]:
#pour chaque dictionnaire on sépare display_name en deux listes, on garde le premier élement qui correspond au lieu
for i in range(len(liste)):
    liste[i][0]['display_name']=liste[i][0]['display_name'].split(',')[0]

In [6]:
#check ok
liste[5][0]['display_name'].split(',')[0]

'Paris'

In [7]:
#création dataframe avec une list comprehension 
df_cities=pd.DataFrame(liste[i][0] for i in range(len(liste)))

In [8]:
df_cities.head(6)

,place_id,licence,osm_type,osm_id,boundingbox,lat,lon,display_name,class,type,...,city,suburb,city_district,historic,road,isolated_dwelling,state_district,tourism,neighbourhood,boundary
0,151486647,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,211285890,"[48.6349172, 48.637031, -1.5133292, -1.5094796]",48.6359541,-1.511459954959514,Mont Saint-Michel,place,islet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,282098015,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,905534,"[48.5979853, 48.6949736, -2.0765246, -1.9367259]",48.649518,-2.0260409,Saint-Malo,boundary,administrative,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,281962470,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,145776,"[49.2608124, 49.2934736, -0.7275671, -0.6757378]",49.2764624,-0.7024738,Bayeux,boundary,administrative,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,282341149,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,104492,"[49.4516697, 49.5401463, 0.0667992, 0.1955556]",49.4938975,0.1079732,Le Havre,boundary,administrative,...,Le Havre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,122848,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,26686587,"[49.2804591, 49.6004591, 0.9339658, 1.2539658]",49.4404591,1.0939658,Rouen,place,city,...,Rouen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,281739181,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,7444,"[48.8155755, 48.902156, 2.224122, 2.4697602]",48.8588897,2.3200410217200766,Paris,boundary,administrative,...,Paris,Paris,Paris,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Le but du code suivant est de faire passer le display_name en première colonne de df_cities
cols = df_cities.columns.tolist()
cols_pop=cols.pop(7)
cols.append(cols_pop)
cols = cols[-1:] + cols[:-1]
df_cities=df_cities[cols]
df_cities.head(1)

,display_name,place_id,licence,osm_type,osm_id,boundingbox,lat,lon,class,type,...,city,suburb,city_district,historic,road,isolated_dwelling,state_district,tourism,neighbourhood,boundary
0,Mont Saint-Michel,151486647,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,211285890,"[48.6349172, 48.637031, -1.5133292, -1.5094796]",48.6359541,-1.511459954959514,place,islet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df=df_cities[['display_name', 'lat','lon']]

In [11]:
df=df.rename(columns={'display_name':'City'})
df.insert(loc=0, column='Id', value=range(1,len(df)+1))

In [12]:
df

,Id,City,lat,lon
0,1,Mont Saint-Michel,48.6359541,-1.511459954959514
1,2,Saint-Malo,48.649518,-2.0260409
2,3,Bayeux,49.2764624,-0.7024738
3,4,Le Havre,49.4938975,0.1079732
4,5,Rouen,49.4404591,1.0939658
5,6,Paris,48.8588897,2.3200410217200766
6,7,Amiens,49.8941708,2.2956951
7,8,Lille,50.6365654,3.0635282
8,9,Strasbourg,48.584614,7.7507127
9,10,Château du Haut-Kœnigsbourg,48.249489800000006,7.34429620253195


In [13]:
# Récupération des données de prévisions météo journalières sur les 7 prochains jours 
liste2=[]
for i in tqdm(range(len(cities))):
    para={'lat':df.loc[i,'lat'],
          'lon':df.loc[i,'lon'],
          'appid':api_key,
          'exclude': 'current,hourly,minutely',
          'units':'metric',}
    r=requests.get('https://api.openweathermap.org/data/2.5/onecall', params=para)
    liste2.append(r.json())

100%|██████████| 35/35 [00:03<00:00,  8.94it/s]


In [14]:
#liste2[7]

In [15]:
# Calcul pour chaque ville de la la temperature du jour moyenne sur les 7 prochains jours 
for i in range(len(cities)):
    temperature= pd.Series(day['temp']['day']for day in liste2[i]['daily'])
    temperature_mean=temperature.mean()
    df.loc[i, 'avg_day_temperature'] = temperature_mean

In [16]:
# Calcul pour chaque ville de la la probabilité de pluie du jour moyenne sur les 7 prochains jours 
for i in range(len(cities)):
    pop= pd.Series(day['pop']for day in liste2[i]['daily'])
    pop_mean=pop.mean()
    df.loc[i, 'avg_rain_probability'] = pop_mean

In [17]:
#for i in range(len(cities)):
#    feels_like= pd.Series(day['feels_like']['day']for day in liste2[i]['daily'])
#    feels_like_mean=feels_like.mean()
#    df.loc[i, 'feels_like_day'] = feels_like_mean

In [18]:
#for i in range(len(cities)):
#    humid= pd.Series(day['humidity']for day in liste2[i]['daily'])
#    humid_mean=humid.mean()
#    df.loc[i, 'humidity_mean'] = humid_mean

In [19]:
#for i in range(len(cities)):
#    cloudiness= pd.Series(day['clouds']for day in liste2[i]['daily'])
#    cloudiness_mean=cloudiness.mean()
#    df.loc[i, 'cloudiness_mean'] = cloudiness_mean

In [20]:
#for i in range(len(cities)):
#    weather= pd.Series(day['weather'][0]['main']for day in liste2[i]['daily'])
#    weather_mode=weather.mode()[0]
#    df.loc[i, 'main_weather'] = weather_mode

In [21]:
df

,Id,City,lat,lon,avg_day_temperature,avg_rain_probability
0,1,Mont Saint-Michel,48.6359541,-1.511459954959514,7.87375,0.73625
1,2,Saint-Malo,48.649518,-2.0260409,8.78750,0.75000
2,3,Bayeux,49.2764624,-0.7024738,7.38000,0.87250
3,4,Le Havre,49.4938975,0.1079732,7.58375,0.84750
4,5,Rouen,49.4404591,1.0939658,6.26000,0.73875
5,6,Paris,48.8588897,2.3200410217200766,5.69375,0.60625
6,7,Amiens,49.8941708,2.2956951,5.22500,0.66125
7,8,Lille,50.6365654,3.0635282,5.11875,0.67375
8,9,Strasbourg,48.584614,7.7507127,3.87125,0.70000
9,10,Château du Haut-Kœnigsbourg,48.249489800000006,7.34429620253195,0.50500,0.76000


In [22]:
#df.insert(loc=0, column='Id', value=range(1,len(df)+1))

In [23]:
#df.drop('place_id', axis=1)

In [24]:
# Sauvegarde du df au format csv
df.to_csv("cities_and_weather.csv", index=False)
#ligne suivante à garder si besoin de garder un historique (avec timestamp)
#df.to_csv("cities_and_weather_"+str(dt.datetime.today()).replace('-','').replace(' ','_')[:11]+".csv", index=False)

In [25]:
df_by_avgpop_asc = df.sort_values(by=['avg_rain_probability'], ascending=True).reset_index(drop=True)
df_by_avgpop_asc.head()

,Id,City,lat,lon,avg_day_temperature,avg_rain_probability
0,18,Gorges du Verdon,43.7496562,6.3285616,3.00875,0.29250
1,27,Saintes-Maries-de-la-Mer,43.4522771,4.4287172,9.17125,0.30250
2,20,Cassis,43.2140359,5.5396318,8.94875,0.30625
3,22,Aix-en-Provence,43.5298424,5.4474738,6.46625,0.31125
4,21,Marseille,43.2961743,5.3699525,8.69500,0.31375


In [26]:
# Sauvegarde de df_by_pop_mean au format csv
df_by_avgpop_asc.to_csv("cities_avg_rain_probability_asc.csv", index=False)
#ligne suivante à garder si besoin de garder un historique (avec timestamp)
#df_by_avgpop_asc.to_csv("cities_avg_pop_asc_"+str(dt.datetime.today()).replace('-','').replace(' ','_')[:11]+".csv", index=False)

In [27]:
df_by_avgtemp_desc = df.sort_values(by=['avg_day_temperature'], ascending=False).reset_index(drop=True)
df_by_avgtemp_desc.head()

,Id,City,lat,lon,avg_day_temperature,avg_rain_probability
0,28,Collioure,42.52505,3.0831554,10.89500,0.37625
1,34,Bayonne,43.4933379,-1.475099,9.80375,0.85875
2,33,Biarritz,43.471143749999996,-1.552726590666314,9.58875,0.86875
3,26,Aigues-Mortes,43.5658225,4.1912837,9.24875,0.32500
4,27,Saintes-Maries-de-la-Mer,43.4522771,4.4287172,9.17125,0.30250


In [28]:
# Sauvegarde de df_by_pop_mean au format csv
df_by_avgtemp_desc.to_csv("cities_avg_day_temperature_desc.csv", index=False)
#ligne suivante à garder si besoin de garder un historique (avec timestamp)
#df_by_avgtemp_desc.to_csv("cities_avg_temp_desc_"+str(dt.datetime.today()).replace('-','').replace(' ','_')[:11]+".csv", index=False)